## Cофья Скобелева
# Самостоятельная работа 3, ансамбли моделей

В этом задании мы с вами немного коснёмся метеорологии, чтобы предсказать, будет ли дождь в Австралии. Попутно вы узнаете, какие факторы влияют на вероятность дождя — возможно, вы научитесь предсказывать его точнее, чем метеорологические службы, и больше никогда не окажетесь в нужный момент без зонта.

Данные содержат 23 признака и 145 460 наблюдений. Из этих 23 признаков 6 — категориальные, в одном записана дата, а остальные являются непрерывными числовыми данными.

Примеры числовых признаков: температура, скорость ветра, влажность, облачность, атмосферное давление в разное время суток, количество осадков, испарение, количество часов с солнечной погодой.
Примеры категориальных признаков: местоположение, направление ветра в разное время суток, наличие дождя сегодня или завтра.

Целевой переменной является столбец RainTomorrow. Значение этой переменной мы и будем пытаться предсказать.

In [94]:
import pandas as pd
df = pd.read_csv('WeatherAUS.csv')
df.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

## Задание 1 (0.5 балла)

Для начала сделаем небольшую предобработку данных. Сколько всего в данных пропущенных значений?

In [96]:
df.isna().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

## Задание 2 (1.5 балла)

Проведём небольшой разведывательный анализ.

__1. Для какой локации больше всего наблюдений? Найдите усредненное количество осадков для нее.__

In [97]:
df['Location'].describe()

count       145460
unique          49
top       Canberra
freq          3436
Name: Location, dtype: object

In [98]:
df[df['Location']=='Canberra']['Rainfall'].mean()

1.7417203042715037

Больше всего наблюдений для локации Canberra, усредненное количество осадков ≈ 1.74

__2. Выше средняя облачность в 9 утра или в 3 часа дня?__

In [99]:
df['Cloud9am'].mean()

4.4474612602152455

In [100]:
df['Cloud3pm'].mean()

4.509930082924903

Средняя облачность выше в 3 часа дня

__3. Сравните количество дней с дождями и без (для обеих колонок про дождь)__

In [101]:
rain_today = df[df['RainToday']=='Yes']['RainToday'].count()
no_rain_today = df[df['RainToday']=='No']['RainToday'].count()

rain_tomorrow = df[df['RainTomorrow']=='Yes']['RainTomorrow'].count()
no_rain_tomorrow = df[df['RainTomorrow']=='No']['RainTomorrow'].count()

print(f"Количество дней с дождем: Сегодня - {rain_today}, Завтра - {rain_tomorrow}")
print(f"Количество дней без дождя: Сегодня - {no_rain_today}, Завтра - {no_rain_tomorrow}")

Количество дней с дождем: Сегодня - 31880, Завтра - 31877
Количество дней без дождя: Сегодня - 110319, Завтра - 110316


Дней с дождем для обеих колонок (сегодня и завтра) меньше, чем дней без дождя

## Задание 3 (1 балл)

В некоторых признаках пропусков более 40 % — удалите такие признаки. Сколько их было?

In [102]:
df.shape

(145460, 23)

In [103]:
df.isna().sum()/df.shape[0]*100

Date              0.000000
Location          0.000000
MinTemp           1.020899
MaxTemp           0.866905
Rainfall          2.241853
Evaporation      43.166506
Sunshine         48.009762
WindGustDir       7.098859
WindGustSpeed     7.055548
WindDir9am        7.263853
WindDir3pm        2.906641
WindSpeed9am      1.214767
WindSpeed3pm      2.105046
Humidity9am       1.824557
Humidity3pm       3.098446
Pressure9am      10.356799
Pressure3pm      10.331363
Cloud9am         38.421559
Cloud3pm         40.807095
Temp9am           1.214767
Temp3pm           2.481094
RainToday         2.241853
RainTomorrow      2.245978
dtype: float64

Более 40% пропусков у трех признаков: `Evaporation` (43.17), `Sunshine`(48.01), `Cloud3pm` (40.81). Удалим эти признаки

In [104]:
df.drop(['Evaporation', 'Sunshine', 'Cloud3pm'], axis = 1, inplace = True)

Также удалите все строки с пропусками:

In [105]:
df.dropna(inplace=True)

In [106]:
df.isna().sum()

Date             0
Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
dtype: int64

## Задание 4 (1 балл)

Для  категориальных признаков создайте dummy-переменные. 

Категориальные признаки: Location, WindGustDir, WindDir9am, WindDir3pm, RainToday, RainTomorrow

In [107]:
df['RainToday'] = df['RainToday'].map({'No':0 , 'Yes':1})
df['RainTomorrow'] = df['RainTomorrow'].map({'No':0 , 'Yes':1})

In [108]:
df = pd.get_dummies(df, columns=['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm'])

In [109]:
df.shape

(75615, 98)

In [110]:
df

,Date,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,...,WindDir3pm_NNW,WindDir3pm_NW,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW
0,2008-12-01,13.4,22.9,0.6,44.0,20.0,24.0,71.0,22.0,1007.7,...,0,0,0,0,0,0,0,0,1,0
4,2008-12-05,17.5,32.3,1.0,41.0,7.0,20.0,82.0,33.0,1010.8,...,0,1,0,0,0,0,0,0,0,0
6,2008-12-07,14.3,25.0,0.0,50.0,20.0,24.0,49.0,19.0,1009.6,...,0,0,0,0,0,0,0,1,0,0
11,2008-12-12,15.9,21.7,2.2,31.0,15.0,13.0,89.0,91.0,1010.5,...,0,0,0,0,0,0,0,0,0,0
12,2008-12-13,15.9,18.6,15.6,61.0,28.0,28.0,76.0,93.0,994.3,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145428,2017-05-25,14.6,26.3,0.0,37.0,19.0,20.0,61.0,36.0,1022.0,...,0,0,1,0,0,0,0,0,0,0
145431,2017-05-28,8.0,24.6,0.0,33.0,11.0,13.0,46.0,25.0,1021.7,...,0,0,0,0,0,0,0,0,0,0
145432,2017-05-29,12.7,22.2,0.0,37.0,19.0,13.0,59.0,34.0,1024.3,...,0,0,0,0,0,0,0,0,0,0
145433,2017-05-30,9.4,22.7,0.0,35.0,13.0,17.0,62.0,32.0,1026.6,...,0,0,0,0,0,0,0,0,0,0


## Задание 5 (1 балл)

Разбейте данные на обучающую и тестовую выборки в соотношении 70/30, в качестве значения параметра random_state возьмите число 31.

In [111]:
from sklearn.model_selection import train_test_split

X = df.drop(['RainTomorrow', 'Date'], axis=1)
y = df['RainTomorrow']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=31)

## Задание 6 (1 балл)

Теперь можно перейти к обучению прогностических моделей. Начнём с того, что построим простейшую логистическую регрессию (без настройки гиперпараметров). Это будет та модель, с качеством которой мы будем сравнивать результаты, полученные далее, чтобы оценить превосходство случайного леса над простыми методами.

Оцените ее качество как минимум с помощью 2 метрик.

In [124]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train) 
X_test = scaler.transform(X_test)

In [129]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model_lr = LogisticRegression()
model_lr.fit(X_train, y_train)
preds_test = model_lr.predict(X_test)

accuracy = accuracy_score(y_test, preds_test)
f1 = f1_score(y_test, preds_test)
precision = precision_score(y_test, preds_test)
recall = recall_score(y_test, preds_test)
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-score: {f1}')

Accuracy: 0.8522371611196826
Precision: 0.7387883556254917
Recall: 0.5445582833945486
F1-score: 0.6269752949031827


## Задание 7 (2.5 балла)

Перейдем к следующему алгоритму — градиентному бустингу.

Будем настраивать количество деревьев и темп обучения, делая перебор по следующей сетке:

params = {"n_estimators": 2**np.arange(4), "learning_rate": 0.1**np.arange(2)}
Используйте для поиска оптимальных параметров GridSearchCV, а для ускорения работы алгоритма задайте параметр кросс-валидации, равный 3.

Какое наибольшее значение метрики f1_score получилось? 

In [120]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

In [126]:
import numpy as np

model_for_gs = GradientBoostingClassifier()
params = {"n_estimators": 2**np.arange(4), "learning_rate": 0.1**np.arange(2)}
gs = GridSearchCV(model_for_gs, 
                  params, 
                  cv=3, 
                  scoring=make_scorer(f1_score),
                  verbose=5)
gs.fit(X_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV 1/3] END .learning_rate=1.0, n_estimators=1;, score=0.557 total time=   0.2s
[CV 2/3] END .learning_rate=1.0, n_estimators=1;, score=0.547 total time=   0.1s
[CV 3/3] END .learning_rate=1.0, n_estimators=1;, score=0.566 total time=   0.1s
[CV 1/3] END .learning_rate=1.0, n_estimators=2;, score=0.579 total time=   0.3s
[CV 2/3] END .learning_rate=1.0, n_estimators=2;, score=0.585 total time=   0.3s
[CV 3/3] END .learning_rate=1.0, n_estimators=2;, score=0.575 total time=   0.3s
[CV 1/3] END .learning_rate=1.0, n_estimators=4;, score=0.585 total time=   0.5s
[CV 2/3] END .learning_rate=1.0, n_estimators=4;, score=0.591 total time=   0.5s
[CV 3/3] END .learning_rate=1.0, n_estimators=4;, score=0.595 total time=   0.5s
[CV 1/3] END .learning_rate=1.0, n_estimators=8;, score=0.600 total time=   1.0s
[CV 2/3] END .learning_rate=1.0, n_estimators=8;, score=0.600 total time=   1.0s
[CV 3/3] END .learning_rate=1.0, n_estimators=8;,

GridSearchCV(cv=3, estimator=GradientBoostingClassifier(),
             param_grid={'learning_rate': array([1. , 0.1]),
                         'n_estimators': array([1, 2, 4, 8])},
             scoring=make_scorer(f1_score), verbose=5)

In [127]:
print("Лучшие гиперпараметры:", gs.best_params_)
print("Лучшее значение метрики:", gs.best_score_)

Лучшие гиперпараметры: {'learning_rate': 1.0, 'n_estimators': 8}
Лучшее значение метрики: 0.6003160321298036


Наибольшее значение метрики f1_score получилось равным примерно 0.60, что немного хуже по сравнению с результатом логистической регрессии (примерно 0.63)

## Задание 8 (1.5 балл)

Обучите алгоритм XGBoost. Так как он достаточно мощный «из коробки», определите его с параметрами по умолчанию, только задайте random_state = 26. Какое значение метрики f1_score получилось? 

In [128]:
from xgboost import XGBClassifier

model_xgb = XGBClassifier(random_state=26)
model_xgb.fit(X_train,y_train)
preds_test = model_xgb.predict(X_test)
f1_score(preds_test, y_test)

0.6570100529672468

Значение метрики f1_score получилось равным примерно 0.66, XGBoost работает лучше по сравнению с логистической регрессией и градиентным бустингом